In [12]:
import os
import cv2
import matplotlib.pyplot as plt


pasta_origem = "C:/Users/lazar/OneDrive/Documentos/Iniciacao Cientifica/Processamento-Digital-De-Imagens/CgToll10B"

pasta_destino = "C:/Users/lazar/OneDrive/Documentos/Iniciacao Cientifica/Processamento-Digital-De-Imagens/suavizacao_e_otsu"

arq = os.listdir(pasta_origem)

for i in arq:
    #verificando se o que está sendo lido é uma imagem desejada
    if i.lower().startswith(('cgtoll10b')):
        caminho_origem = os.path.join(pasta_origem, i)
        img = cv2.imread(caminho_origem, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Erro ao ler a imagem: {caminho_origem}")
        else:
            print(f"Imagem lida com sucesso: {caminho_origem}")

            gaussiana = cv2.GaussianBlur(img, (3, 3), 1)
            _, otsu = cv2.threshold(gaussiana, 175, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

            cv2.imwrite(os.path.join(pasta_destino, f"{i}"), otsu)


Imagem lida com sucesso: C:/Users/lazar/OneDrive/Documentos/Iniciacao Cientifica/Processamento-Digital-De-Imagens/CgToll10B\CgToll10B_RabbitTube546_ Dorsal647_03_06_2019.lif_Series009_z0_ch00.tif
Imagem lida com sucesso: C:/Users/lazar/OneDrive/Documentos/Iniciacao Cientifica/Processamento-Digital-De-Imagens/CgToll10B\CgToll10B_RabbitTube546_ Dorsal647_03_06_2019.lif_Series009_z0_ch01.tif
Imagem lida com sucesso: C:/Users/lazar/OneDrive/Documentos/Iniciacao Cientifica/Processamento-Digital-De-Imagens/CgToll10B\CgToll10B_RabbitTube546_ Dorsal647_03_06_2019.lif_Series009_z0_ch02.tif
Imagem lida com sucesso: C:/Users/lazar/OneDrive/Documentos/Iniciacao Cientifica/Processamento-Digital-De-Imagens/CgToll10B\CgToll10B_RabbitTube546_ Dorsal647_03_06_2019.lif_Series011_z0_ch00.tif
Imagem lida com sucesso: C:/Users/lazar/OneDrive/Documentos/Iniciacao Cientifica/Processamento-Digital-De-Imagens/CgToll10B\CgToll10B_RabbitTube546_ Dorsal647_03_06_2019.lif_Series011_z0_ch01.tif
Imagem lida com suce